In [19]:
from prophet import Prophet
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os, glob, warnings, datetime as dt
warnings.filterwarnings("ignore")


# List of 50 stock symbols
stocks = [
    "AAPL","MSFT","GOOGL","AMZN","TSLA","META","NVDA","JPM","V","UNH",
    "HD","PG","DIS","MA","BAC","NFLX","ADBE","PYPL","CMCSA","XOM",
    "PFE","KO","INTC","CSCO","NKE","MRK","PEP","ABBV","ABT","CRM",
    "ORCL","T","VZ","CVX","MCD","WMT","DHR","ACN","LLY","AVGO",
    "QCOM","TXN","NEE","COST","TMUS","IBM","SBUX","MDT","HON","AMD"
]

# Folders
processed_folder = "../data/processed"
predictions_folder = "../data/prophet_prediction"
plots_folder = "../plots/prophet_plot"
os.makedirs(predictions_folder, exist_ok=True)
os.makedirs(plots_folder, exist_ok=True)

processed_stocks = []
skipped_stocks = []

for symbol in stocks:
    print(f"\n--- Processing {symbol} ---")
    # Find latest processed CSV for the symbol
    files = glob.glob(f"{processed_folder}/{symbol}_processed*.csv")
    if files:
        processed_file = sorted(files)[-1]  # pick the latest file
    else:
        print(f"Processed file for {symbol} not found. Skipping.")
        skipped_stocks.append(symbol)
        continue
    
    SELECTED_REGRESSORS = [
    'Return_lag1', 
    'Distance_from_MA_20',
    'RSI_14',
    'High_Low_Range',
    'OBV_Divergence'
    ]
    

    # Load data
    data  = pd.read_csv(processed_file, parse_dates=['Price'])
    data = data[['Price', 'Close']].rename(columns={'Price': 'ds', 'Close': 'y'})
    
    # Filter data to include ds, y, and all valid regressors
    valid_regressors = [reg for reg in SELECTED_REGRESSORS if reg in data.columns]
    prophet_data = data[['ds', 'y'] + valid_regressors]
    prophet_data = prophet_data.dropna()
    
    temp = int(len(prophet_data) * 0.8)
    train_data = prophet_data[:temp]
    test_data = prophet_data[temp:]

    train_data['y_scaled'] = train_data['y'] / 100
    test_data['y_scaled'] = test_data['y'] / 100
    
    
    # Create Model
    model_prophet = Prophet(daily_seasonality=True, yearly_seasonality=True,
                            weekly_seasonality=True, interval_width=0.95, changepoint_prior_scale=0.5)
    for reg in valid_regressors:
        model_prophet.add_regressor(reg, prior_scale=0.5, mode='multiplicative')

    model_prophet.fit(train_data)

    # Predict first using test data
    test_future = test_data[['ds']]
    test_predictions = model_prophet.predict(test_future)

    y_true = test_data['y'].values
    y_pred = test_predictions['yhat'].values

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print("\nMetrics:")
    print("Close >> RMSE: $", "%.4f" % rmse, "R2:", "%.4f" % r2, "MAE: $", "%.4f" % mae)

    # Future pred
    future = model_prophet.make_future_dataframe(periods=30)
    predictions = model_prophet.predict(future)
    
    fig = model_prophet.plot(predictions)
    # Add a marker for the split point
    plt.title(f"Prophet Forecast for {symbol}")
    plot_file = os.path.join(plots_folder, f"{symbol}_prophet_plot.png")
    fig.savefig(plot_file)
    plt.close(fig)
    
    # Store data
    predictions_file = os.path.join(predictions_folder, f"{symbol}_predictions.csv")
    predictions.to_csv(predictions_file)



print("\nProphet processing complete.")
print("Processed stocks:", processed_stocks)
print("Skipped stocks:", skipped_stocks)

22:02:25 - cmdstanpy - INFO - Chain [1] start processing



--- Processing AAPL ---


22:02:26 - cmdstanpy - INFO - Chain [1] done processing



Metrics:
Close >> RMSE: $ 47.6359 R2: -1.9704 MAE: $ 40.0941


22:02:27 - cmdstanpy - INFO - Chain [1] start processing


...


Metrics:
Close >> RMSE: $ 49.7559 R2: -14.6162 MAE: $ 44.8401


22:03:35 - cmdstanpy - INFO - Chain [1] start processing



--- Processing AMD ---


22:03:36 - cmdstanpy - INFO - Chain [1] done processing



Metrics:
Close >> RMSE: $ 98.3435 R2: -8.2223 MAE: $ 87.0375

Prophet processing complete.
Processed stocks: []
Skipped stocks: []
